In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver

import time as t

base_url = "http://www.pknu.ac.kr/usrBoardActn.do?p_bm_idx=5&p_boardcode=PK10000005&p_sbsidx=2"

find_data = '장학금'
limit_num = 5
meta_data = []

driver = webdriver.Chrome(
    executable_path="webdriver/chromedriver.exe"
)
driver.get(base_url)
t.sleep(5)

driver.find_element_by_xpath('//*[@id="p_sbsidx"]/option[1]').click()
driver.find_element_by_xpath('//*[@id="cateForm"]/div/input').click()

for i in range(limit_num):
    t.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    meta_content = soup.select('td[class=title]')
    meta_author = soup.select('td[class=author]')
    meta_time = soup.select('td[class=date]')

    for tag1, tag2, tag3 in zip(meta_content, meta_author, meta_time):

        data_title = tag1.text.strip()
        data_author = tag2.text
        data_time = tag3.text
        status = False
        
        for temp in meta_data:
            if data_title == temp[0]:
                status = True
                
        if status:
            continue

        if find_data in data_title:
            print("제목 : ", data_title)
            print("부서 : ", data_author)
            print("날짜 : ", data_time)
            print("페이지 : ", i+1)
            print("-" * 100)
            
        data_author = data_author.replace("\xa0","")
        data_time = data_time.replace("\xa0", "")

        meta_data.append([data_title, data_author, data_time])

    driver.find_element_by_xpath('//*[@id="contents"]/div[4]/div[2]/a[' + str(i + 3) + ']').click()

t.sleep(5)
driver.close()

import requests
# 요청을 위한 상수를 선언합니다: TOKEN은 자신의 것을 입력해주세요.
TARGET_URL = 'https://notify-api.line.me/api/notify'
TOKEN = 'D1WCgxDEY4ZiFzNR3TFNuCGskQomSTubIfRhAujLrSG'
# 요청합니다.

for i in meta_data:
    response = requests.post(
  TARGET_URL,
  headers={
    'Authorization': 'Bearer ' + TOKEN
  },
  data={
    'message':i[0]   
  }
)
# 요청 완료
print(response.text)


제목 :  2020학년도 학부 코로나19 특별장학금 신청 안내(2차)
부서 :  학생복지과 
날짜 :  2020-08-21 
페이지 :  1
----------------------------------------------------------------------------------------------------
제목 :  2020-2학기 중소기업 취업연계 장학금(희망사다리1) 학생 신청 ...
부서 :  학생복지과 
날짜 :  2020-09-14 
페이지 :  1
----------------------------------------------------------------------------------------------------
제목 :  2020학년도 2학기 교내 특별장학금 추가 신청 안내
부서 :  학생복지과 
날짜 :  2020-08-31 
페이지 :  5
----------------------------------------------------------------------------------------------------
제목 :  2020-2학기 고졸 후학습자 장학금(희망사다리Ⅱ) 학생신청 안내
부서 :  학생복지과 
날짜 :  2020-08-31 
페이지 :  5
----------------------------------------------------------------------------------------------------
제목 :  2020-2학기 중소기업 취업연계 장학금(희망사다리1) 학생 신청 안내
부서 :  학생복지과 
날짜 :  2020-08-31 
페이지 :  5
----------------------------------------------------------------------------------------------------


 - *여기서부터 따라하시면 됩니다*

# - 부경대 웹 홈페이지 공지사항을 전해주는 Line 챗봇을 만들어보자

약 2년간 생각했던 프로젝트가 졸업하고 나서야 이제야 완성하네요...

지금까지 부경대학교 공지사항을 추출하고 정리해서 엑셀파일로 취합하고 검색 과정까지 구현하였는데요<br>
이렇게까지 해도 신기하고 편리한 과정이지만<br>
그러면 매번 파이썬 프로그램을 실행해서 봐야하나...? 라고 의문이 드실 수도 있으실 겁니다

사실 더 편리하게 누구보다 빠르게 소식을 접하려면 모바일이 최곱니다<br>
저도 옛날부터 그렇게 생각해서 이 프로젝트의 최종점은 어플에서 받아보는 형태가 될 것이라고 예상했었구요

---

따라서 이번 시간에는 부경대 웹 홈페이지에서 공지사항을 긁어와서 정리한 다음<br>
Line 어플 내에 챗봇을 만들어서 일정 시간마다 메세지를 보내주는 프로그램을 작성해봅시다

참고로 메세지만 유저에게 전달하는 형태이기 때문에 우선 Line Notify로 구현한 다음<br>
추가적으로 더 기능을 구현하고자 할 때 Line 챗봇 순으로 진행하겠습니다